In [ ]:
import os
import json
from platform import system

import ollama
import gradio as gr
from Tools.scripts.dutree import display



In [ ]:
MODEL='llama3.2:latest'

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. Give short, courteous answers, no more than 1 sentence. Always be accurate. If you don't know the answer, say so."

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = ollama.chat(model=MODEL, messages=messages)
    return response.message.content

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("London")

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": price_function}]

In [ ]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_message}
    ] + history + [
        {"role": "user", "content": message}
    ]
    response = ollama.chat(
        model=MODEL, messages=messages, tools=tools
    )

    if response.get("done_reason") == "stop":
        tool_call_message = response["message"]
        tool_response, city = handle_tool_call(tool_call_message)
        messages.append(tool_call_message)
        messages.append(tool_response)
        response = ollama.chat(
            model=MODEL, messages=messages
        )
    return response.message.content

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = tool_call.function.arguments
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({
            "destination_city": city,
            "price": price
        }),
    }

    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

## let's go multi-modal

In [ ]:
from PIL import Image
import torch
from diffusers import StableDiffusionPipeline
import base64
from io import BytesIO
from IPython.display import display
from diffusers import AutoPipelineForText2Image

In [ ]:
IMAGE_MODEL = "stabilityai/stable-diffusion-xl-base-1.0"
def artist(city):
    # Initialize the Stable Diffusion pipeline for image generation
    pipe = AutoPipelineForText2Image.from_pretrained(
        IMAGE_MODEL,  # Specify the model to use (e.g., "stabilityai/stable-diffusion-xl-base-1.0")
        torch_dtype=torch.float16,  # Use half-precision (float16) for faster computation and reduced memory usage
        use_safetensors=True,  # Use the safer `.safetensors` format for loading model weights
        variant="fp16",  # Specify the variant of the model optimized for float16 precision
    ).to("cuda")  # Move the pipeline to the GPU for faster computation

    # Enable CPU offloading for faster image generation
    pipe.enable_model_cpu_offload()
    pipe.enable_xformers_memory_efficient_attention() # Enable memory-efficient attention for faster computation and reduced memory usage

    prompt = f"A vibrant pop-art style illustration of {city} vacation, highlighting famous tourists attractions and unique cultural elements, bold colors, comic book aesthetic, high contrast, and dynamic composition."

    # Generate an image using the Stable Diffusion XL pipeline
    image = pipe(
        prompt=prompt,
    ).images[0] # Extract the first image from the output list

    return image

In [ ]:
image = artist("Paris")
display(image)